In [1]:
from config import *
from utilities import get_model_name
from dataset import ClassificationDataset

import functools
import numpy as np
import pandas as pd

results_dict = {}
internal, external = 11, 12
encode_method = "dummy"
impute_method = "knn7d" #TODO
fs_method, fs_ratio = "dnp", 0.5  #dnp & graces
norm_method = "maxmin"
classification_dataset = ClassificationDataset(
    internal, external, encode_method, impute_method, fs_method, fs_ratio, norm_method, random_state=SEED)

Number of Selected Features: 100%|██████████| 71/71 [06:04<00:00,  5.13s/it]


In [2]:
# from sklearn.neural_network import MLPClassifier
from utilities import MLPClassifier

model_fn = functools.partial(
    MLPClassifier, random_state=SEED, max_iter=50,
    momentum=0.9, squares=0.999, optimizer_t='sgdm',
    use_residual=True, use_batch_norm=False)
param_grid_list = [{
    # hidden_layer_sizes = (64, 32)
    # config       lr_range            worst_auc
    # adam w/ bn:  [2e-5, 5e-5, 1e-4]  0.8942
    # adam w/o bn: [5e-5, 1e-4, 2e-4]  0.9148
    # sgdm w/ bn:  [1e-3, 2e-3, 5e-3]  0.8982
    # sgdm w/o bn: [5e-3, 1e-2, 2e-2]  0.9177

    'hidden_layer_sizes': [(128, 128), (64, 64, 64)],
    'learning_rate': [5e-3, 1e-2, 2e-2],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

MLPClassifier


Epoch 11/50:   0%|          | 0/26 [00:00<?, ?batch/s, loss=0.164]          

model(hidden_layer_sizes=(128, 128), learning_rate=0.005)
Valid:         AUC: 0.9176 | AUPR: 0.6539 | Acc: 95.36%


model(hidden_layer_sizes=(128, 128), learning_rate=0.01)
Valid:         AUC: 0.9211 | AUPR: 0.6609 | Acc: 95.37%


model(hidden_layer_sizes=(128, 128), learning_rate=0.02)
Valid:         AUC: 0.9231 | AUPR: 0.6627 | Acc: 95.22%


model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.005)
Valid:         AUC: 0.9175 | AUPR: 0.6566 | Acc: 95.40%


model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.01)
Valid:         AUC: 0.9218 | AUPR: 0.6644 | Acc: 95.40%


model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.02)
Valid:         AUC: 0.9215 | AUPR: 0.6660 | Acc: 95.33%
best model(hidden_layer_sizes=(128, 128), learning_rate=0.02)
Valid:         AUC: 0.9231 | AUPR: 0.6627 | Acc: 95.22%
Internal test: AUC: 0.9282 | AUPR: 0.6965 | Acc: 95.09%
External test: AUC: 0.9220 | AUPR: 0.6600 | Acc: 95.04%


In [3]:
from sklearn.svm import SVC

model_fn = functools.partial(
    SVC, random_state=SEED, probability=True,
    class_weight='balanced')
param_grid_list = [{
    'kernel': ['linear'],
    'C': [0.01, 0.1, 1],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

SVC


model(kernel=linear, C=0.01)
Valid:         AUC: 0.9174 | AUPR: 0.6229 | Acc: 94.95%
model(kernel=linear, C=0.1)
Valid:         AUC: 0.9172 | AUPR: 0.6162 | Acc: 94.93%
model(kernel=linear, C=1)
Valid:         AUC: 0.9168 | AUPR: 0.6181 | Acc: 95.01%
best model(kernel=linear, C=0.01)
Valid:         AUC: 0.9174 | AUPR: 0.6229 | Acc: 94.95%
Internal test: AUC: 0.9233 | AUPR: 0.6613 | Acc: 94.70%
External test: AUC: 0.9159 | AUPR: 0.6097 | Acc: 94.63%


In [4]:
from sklearn.neighbors import KNeighborsClassifier

model_fn = functools.partial(
    KNeighborsClassifier, n_jobs=-1, algorithm='auto', p=2)
param_grid_list = [{
    'n_neighbors': [3, 5, 7],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

KNeighborsClassifier
model(n_neighbors=3)
Valid:         AUC: 0.7518 | AUPR: 0.4232 | Acc: 94.70%
model(n_neighbors=5)
Valid:         AUC: 0.7780 | AUPR: 0.4657 | Acc: 94.96%
model(n_neighbors=7)
Valid:         AUC: 0.7976 | AUPR: 0.4914 | Acc: 95.00%
best model(n_neighbors=7)
Valid:         AUC: 0.7976 | AUPR: 0.4914 | Acc: 95.00%
Internal test: AUC: 0.8110 | AUPR: 0.5444 | Acc: 94.99%
External test: AUC: 0.8056 | AUPR: 0.5054 | Acc: 94.85%


In [5]:
from sklearn.naive_bayes import GaussianNB

model_fn = GaussianNB
param_grid_list = [{}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

GaussianNB
model()
Valid:         AUC: 0.8788 | AUPR: 0.4570 | Acc: 83.20%
best model()
Valid:         AUC: 0.8788 | AUPR: 0.4570 | Acc: 83.20%
Internal test: AUC: 0.8867 | AUPR: 0.5094 | Acc: 83.94%
External test: AUC: 0.8764 | AUPR: 0.4459 | Acc: 82.32%


In [6]:
# from sklearn.exceptions import ConvergenceWarning
# import warnings
# warnings.filterwarnings("ignore", category=ConvergenceWarning)
from sklearn.linear_model import LogisticRegression

model_fn = functools.partial(
    LogisticRegression, random_state=SEED, n_jobs=-1, 
    max_iter=2000, solver='saga', penalty='l2',
    )
param_grid_list = [{
    'C': [1.0, 10.0, 100.0],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(
    model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(
    best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

LogisticRegression


model(C=1.0)
Valid:         AUC: 0.9190 | AUPR: 0.6526 | Acc: 95.27%
model(C=10.0)
Valid:         AUC: 0.9188 | AUPR: 0.6516 | Acc: 95.27%
model(C=100.0)
Valid:         AUC: 0.9187 | AUPR: 0.6515 | Acc: 95.27%
best model(C=1.0)
Valid:         AUC: 0.9190 | AUPR: 0.6526 | Acc: 95.27%
Internal test: AUC: 0.9251 | AUPR: 0.6856 | Acc: 95.13%
External test: AUC: 0.9144 | AUPR: 0.6341 | Acc: 94.89%


In [7]:
from sklearn.tree import DecisionTreeClassifier

model_fn = functools.partial(
    DecisionTreeClassifier, random_state=SEED,
    ccp_alpha=0.0, class_weight='balanced', max_depth=5)
param_grid_list = [{
        'criterion': ['gini', 'log_loss', 'entropy'],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

DecisionTreeClassifier


model(criterion=gini)
Valid:         AUC: 0.8711 | AUPR: 0.5587 | Acc: 83.80%
model(criterion=log_loss)
Valid:         AUC: 0.8705 | AUPR: 0.5717 | Acc: 81.50%
model(criterion=entropy)
Valid:         AUC: 0.8705 | AUPR: 0.5717 | Acc: 81.50%
best model(criterion=gini)
Valid:         AUC: 0.8711 | AUPR: 0.5587 | Acc: 83.80%
Internal test: AUC: 0.8845 | AUPR: 0.6106 | Acc: 84.45%
External test: AUC: 0.8872 | AUPR: 0.5884 | Acc: 84.36%


In [8]:
from sklearn.ensemble import RandomForestClassifier

model_fn = functools.partial(
    RandomForestClassifier, random_state=SEED, n_jobs=-1,
    ccp_alpha=0.0, class_weight='balanced', max_depth=5, bootstrap=False)
param_grid_list = [{
        'criterion': ['gini', 'log_loss', 'entropy'],
        'n_estimators': [50, 100, 200],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

RandomForestClassifier
model(criterion=gini, n_estimators=50)
Valid:         AUC: 0.9137 | AUPR: 0.6524 | Acc: 89.18%
model(criterion=gini, n_estimators=100)
Valid:         AUC: 0.9126 | AUPR: 0.6488 | Acc: 89.40%
model(criterion=gini, n_estimators=200)
Valid:         AUC: 0.9132 | AUPR: 0.6504 | Acc: 89.41%
model(criterion=log_loss, n_estimators=50)
Valid:         AUC: 0.9128 | AUPR: 0.6539 | Acc: 89.43%
model(criterion=log_loss, n_estimators=100)
Valid:         AUC: 0.9123 | AUPR: 0.6503 | Acc: 89.51%
model(criterion=log_loss, n_estimators=200)
Valid:         AUC: 0.9132 | AUPR: 0.6517 | Acc: 89.54%
model(criterion=entropy, n_estimators=50)
Valid:         AUC: 0.9128 | AUPR: 0.6539 | Acc: 89.43%
model(criterion=entropy, n_estimators=100)
Valid:         AUC: 0.9123 | AUPR: 0.6503 | Acc: 89.51%
model(criterion=entropy, n_estimators=200)
Valid:         AUC: 0.9132 | AUPR: 0.6517 | Acc: 89.54%
best model(criterion=gini, n_estimators=50)
Valid:         AUC: 0.9137 | AUPR: 0.6524 | Acc: 89

In [9]:
from sklearn.ensemble import GradientBoostingClassifier

model = functools.partial(
    GradientBoostingClassifier, random_state=SEED,
    learning_rate=0.1, max_depth=5, loss='log_loss', 
    n_estimators=100)
param_grid_list = [{
        'criterion': ['friedman_mse', 'squared_error'],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

GradientBoostingClassifier


model(criterion=friedman_mse)
Valid:         AUC: 0.9183 | AUPR: 0.6424 | Acc: 95.19%
model(criterion=squared_error)
Valid:         AUC: 0.9183 | AUPR: 0.6426 | Acc: 95.21%
best model(criterion=squared_error)
Valid:         AUC: 0.9183 | AUPR: 0.6426 | Acc: 95.21%
Internal test: AUC: 0.9242 | AUPR: 0.6985 | Acc: 95.30%
External test: AUC: 0.9211 | AUPR: 0.6638 | Acc: 95.21%


In [10]:
from sklearn.ensemble import AdaBoostClassifier
model = functools.partial(AdaBoostClassifier, random_state=SEED, algorithm="SAMME")
param_grid_list = [{
    'n_estimators': [100, 200, 500],
    'learning_rate': [1.0],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

AdaBoostClassifier
model(n_estimators=100, learning_rate=1.0)
Valid:         AUC: 0.9145 | AUPR: 0.6199 | Acc: 95.04%
model(n_estimators=200, learning_rate=1.0)
Valid:         AUC: 0.9155 | AUPR: 0.6227 | Acc: 95.05%
model(n_estimators=500, learning_rate=1.0)
Valid:         AUC: 0.9159 | AUPR: 0.6240 | Acc: 95.01%
best model(n_estimators=500, learning_rate=1.0)
Valid:         AUC: 0.9159 | AUPR: 0.6240 | Acc: 95.01%
Internal test: AUC: 0.9207 | AUPR: 0.6847 | Acc: 95.30%
External test: AUC: 0.9161 | AUPR: 0.6183 | Acc: 94.67%


In [11]:
dataframe = pd.DataFrame(results_dict).transpose().reset_index()
dataframe.columns = ['model', 'val_auc', 'val_aupr', 'val_acc', 'intest_auc', 'intest_aupr', 'intest_acc', 'extest_auc', 'extest_aupr', 'extest_acc']
# dataframe.to_csv()
filename = os.path.join(RESULTS_DIR, encode_method, impute_method, f'{fs_method}{int(fs_ratio * 142)}.csv')
dirname = os.path.dirname(filename)
if not os.path.exists(dirname):
    os.makedirs(dirname)
dataframe.to_csv(filename, index=False)